In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql.functions import regexp_extract, col, when

In [4]:
spark = SparkSession.builder.appName('Clean_event_data').getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/19 12:14:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
events = spark.read.csv('Data/events.csv',header=True,inferSchema=True,sep=';')

In [7]:
events.count()

8710714

In [10]:
events.groupBy('type').count().orderBy('count',ascending=False).show()

+--------------+-------+
|          type|  count|
+--------------+-------+
|          Pass|2434509|
| Ball Receipt*|2293313|
|         Carry|1868736|
|      Pressure| 792485|
| Ball Recovery| 249876|
|          Duel| 187690|
|     Clearance| 110176|
|         Block|  93904|
|       Dribble|  82105|
|   Goal Keeper|  74663|
|Foul Committed|  73087|
|      Foul Won|  69643|
|    Miscontrol|  68593|
|          Shot|  61608|
|  Dispossessed|  60954|
|  Interception|  57124|
| Dribbled Past|  50262|
|  Substitution|  15324|
|      Half End|   9972|
|    Half Start|   9972|
+--------------+-------+
only showing top 20 rows



In [13]:
events_shot = events.filter(events.type == 'Shot')

61608

In [14]:
shot_columns = ['id','match_id','index','player_id','player','position','type','play_pattern','location','under_pressure']
remove_cols = ['goalkeeper_shot_saved_off_target', 'goalkeeper_shot_saved_to_post', 'pass_shot_assist','pass_assisted_shot_id']
# now from events.columns add all columns that have 'shot' in them
shot_columns += [col for col in events.columns if 'shot' in col]
# remove the columns that are not needed
shot_columns = [col for col in shot_columns if col not in remove_cols]

In [15]:
columns_t = ['id','match_id','index','player_id','player','position','type','shot_body_part','play_pattern',
 'shot_type','shot_technique','shot_outcome','shot_statsbomb_xg','location', 'shot_end_location', 'shot_freeze_frame',
 'shot_key_pass_id', 'under_pressure','shot_one_on_one', 'shot_deflected','shot_aerial_won','shot_redirect','shot_follows_dribble',
 'shot_saved_off_target', 'shot_first_time','pass_shot_assist','shot_open_goal',]

In [16]:
events_shot = events.filter(events.type=='Shot').select(shot_columns)
# show without truncating
events_shot.show(5)

+--------------------+--------+-----+---------+--------------------+--------------------+----+-------------+-------------+--------------+---------------+--------------+------------------+---------------+--------------------+--------------------+---------------+------------+-----------------+--------------+---------+--------------+-------------+--------------+------------------+---------------------+--------------------+
|                  id|match_id|index|player_id|              player|            position|type| play_pattern|     location|under_pressure|shot_aerial_won|shot_body_part| shot_end_location|shot_first_time|   shot_freeze_frame|    shot_key_pass_id|shot_one_on_one|shot_outcome|shot_statsbomb_xg|shot_technique|shot_type|shot_open_goal|shot_redirect|shot_deflected|shot_saved_to_post|shot_saved_off_target|shot_follows_dribble|
+--------------------+--------+-----+---------+--------------------+--------------------+----+-------------+-------------+--------------+---------------

In [17]:
cols = events_shot.columns
temp = 'SUM(CASE WHEN {} IS NULL THEN 1 ELSE 0 END) as {}_nulls'
nulls = [temp.format(col,col) for col in cols]
nulls = ','.join(nulls)

In [18]:
events_shot.createOrReplaceTempView('events_shot')

query = f'''
SELECT {nulls}
FROM events_shot
'''

spark.sql(query).show()

+--------+--------------+-----------+---------------+------------+--------------+----------+------------------+--------------+--------------------+---------------------+--------------------+-----------------------+---------------------+-----------------------+----------------------+---------------------+------------------+-----------------------+--------------------+---------------+--------------------+-------------------+--------------------+------------------------+---------------------------+--------------------------+
|id_nulls|match_id_nulls|index_nulls|player_id_nulls|player_nulls|position_nulls|type_nulls|play_pattern_nulls|location_nulls|under_pressure_nulls|shot_aerial_won_nulls|shot_body_part_nulls|shot_end_location_nulls|shot_first_time_nulls|shot_freeze_frame_nulls|shot_key_pass_id_nulls|shot_one_on_one_nulls|shot_outcome_nulls|shot_statsbomb_xg_nulls|shot_technique_nulls|shot_type_nulls|shot_open_goal_nulls|shot_redirect_nulls|shot_deflected_nulls|shot_saved_to_post_nulls|

In [ ]:
events_shot = events_shot.withColumn("shot_location_x", regexp_extract(col("location"), r'\[(.*?),', 1).cast("float")) \
             .withColumn("shot_location_y", regexp_extract(col("location"), r', (.*?)\]', 1).cast("float"))

events_shot.select('location','shot_location_x','shot_location_y').show(5)

In [ ]:
# Split location
# split shot end location # consider ground shots
# solution for shot_freeze_frame
# distribution of xg based on shot type etc
# function to plot all shots
# % of the shot outcome